In [2]:
import mlflow

In [3]:
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-13-222-133-254.compute-1.amazonaws.com:5000/")

In [4]:
# Set or create an experiment
mlflow.set_experiment("02 - Feature Engineering: BoW vs TF-IDF")

2025/11/20 15:28:38 INFO mlflow.tracking.fluent: Experiment with name '02 - Feature Engineering: BoW vs TF-IDF' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://sau-s3-mlflow-bucket/813935114859567786', creation_time=1763670519004, experiment_id='813935114859567786', last_update_time=1763670519004, lifecycle_stage='active', name='02 - Feature Engineering: BoW vs TF-IDF', tags={}>

In [5]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [7]:
df = pd.read_csv(
    '../../data/processed/reddit_clean_final.csv',
    keep_default_na=False,
    na_filter=False
)
df.shape

(36607, 2)

In [8]:
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [9]:

# Step 1: Function to run the experiment
def run_experiment(vectorizer_type, ngram_range, vectorizer_max_features, vectorizer_name):
    # Step 2: Vectorization
    if vectorizer_type == "BoW":
        vectorizer = CountVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"{vectorizer_name}_{ngram_range}_RandomForest")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with {vectorizer_name}, ngram_range={ngram_range}, max_features={vectorizer_max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", vectorizer_type)
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", vectorizer_max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: {vectorizer_name}, {ngram_range}")
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_{vectorizer_name}_{ngram_range}")

# Step 6: Run experiments for BoW and TF-IDF with different n-grams
ngram_ranges = [(1, 1), (1, 2), (1, 3)]  # unigrams, bigrams, trigrams
max_features = 5000  # Example max feature size

for ngram_range in ngram_ranges:
    # BoW Experiments
    run_experiment("BoW", ngram_range, max_features, vectorizer_name="BoW")

    # TF-IDF Experiments
    run_experiment("TF-IDF", ngram_range, max_features, vectorizer_name="TF-IDF")


d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\boto3\compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
2025/11/20 15:37:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 15:37:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 1)_RandomForest at: http://ec2-13-222-133-254.compute-1.amazonaws.com:5000/#/experiments/813935114859567786/runs/cf8f0d445836478c80e5d77a7405df7f
🧪 View experiment at: http://ec2-13-222-133-254.compute-1.amazonaws.com:5000/#/experiments/813935114859567786


2025/11/20 15:38:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 15:38:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 1)_RandomForest at: http://ec2-13-222-133-254.compute-1.amazonaws.com:5000/#/experiments/813935114859567786/runs/d6543eff1e0b4972b0f7881173a913dc
🧪 View experiment at: http://ec2-13-222-133-254.compute-1.amazonaws.com:5000/#/experiments/813935114859567786


2025/11/20 15:39:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 15:40:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\boto3\compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


...

🏃 View run TF-IDF_(1, 2)_RandomForest at: http://ec2-13-222-133-254.compute-1.amazonaws.com:5000/#/experiments/813935114859567786/runs/0fbd4a34aef44bd4bac047e12609e400
🧪 View experiment at: http://ec2-13-222-133-254.compute-1.amazonaws.com:5000/#/experiments/813935114859567786


2025/11/20 15:42:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 15:42:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BoW_(1, 3)_RandomForest at: http://ec2-13-222-133-254.compute-1.amazonaws.com:5000/#/experiments/813935114859567786/runs/e705923b56d941858307074beeeefd40
🧪 View experiment at: http://ec2-13-222-133-254.compute-1.amazonaws.com:5000/#/experiments/813935114859567786


2025/11/20 15:43:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 15:43:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TF-IDF_(1, 3)_RandomForest at: http://ec2-13-222-133-254.compute-1.amazonaws.com:5000/#/experiments/813935114859567786/runs/ec6a4f304722498f919e7a4e18418e19
🧪 View experiment at: http://ec2-13-222-133-254.compute-1.amazonaws.com:5000/#/experiments/813935114859567786
